In [2]:
import Nio as nio
import gzip
from subprocess import call
import numpy as np

In [3]:
print nio.__doc__


PyNIO enables NetCDF-like access for multiple file formats including
    NetCDF 3 (rw) and  GRIB 1 (r), and optionally
    NetCDF 4 (rw), HDF 4 (rw), HDF 5 (rw), HDFEOS 2/4 (r), GRIB 2 (r), HDFEOS 5 (r), shapefiles
    and other GDAL OGR-supported (r) data formats.

import Nio

Class NioFile:

f = Nio.open_file(filepath, mode='r', options=None, history='',format='')

To see summary of file contents, including all attributes :
   print f
attributes:
   name -- the name of the file or group
   dimensions -- dictionary of dimension lengths with dimension name keys
   variables -- dictionary of variable objects with variable name keys
   attributes --  dictionary of global file or group attributes with attribute name keys
       (the following are applicable for advanced formats NetCDF4 and HDF5 only)
   groups -- dictionary of groups with group name keys
   ud_types -- dictionary of user-defined data type definitions with data type name keys
   chunk_dimensions -- dictionary of chunking 

## Gunzip grib2 file

In [6]:
gzfilename = "MRMS_PrecipRate_00.00_20160109-130000.grib2.gz"
call(["gunzip", gzfilename])
filename = gzfilename.strip(".gz")
#f = nio.open_file(filename)
#print(f)

## Open file with nio

In [7]:
f = nio.open_file(filename)
print(f)

Nio file:	MRMS_PrecipRate_00.00_20160109-130000.grib2
   global attributes:
   dimensions:
      lat_0 = 3500
      lon_0 = 7000
   variables:
      float PrecipRate_P0_L102_GLL0 [ lat_0, lon_0 ]
         center :	US NOAA Office of Oceanic and Atmospheric Research
         production_status :	Research products
         long_name :	Radar Precipitation Rate
         units :	mm/hr
         _FillValue :	1e+20
         grid_type :	Latitude/longitude
         parameter_discipline_and_category :	Multi-Radar/Multi-Sensor (MRMS) products, Category 6
         parameter_template_discipline_category_number :	[0, 209, 6, 1]
         level_type :	Specific altitude above mean sea level (m)
         level :	0
         forecast_time :	0
         forecast_time_units :	minutes
         initial_time :	01/09/2016 (13:00)
      float lat_0 [ lat_0 ]
         long_name :	latitude
         grid_type :	Latitude/Longitude
         units :	degrees_north
         Dj :	0.01
         Di :	0.009999999
         Lo2 :

## No groups

In [8]:
print type(f)
print f.groups
print f.attributes

<class 'Nio.NioFile'>
{}
{}


## Look at variables, which is a dictionary

In [9]:
print "dimension keys: \t\n", f.dimensions.keys()
print "dimensions: \t\n", f.dimensions
print "variables: \t\n", f.variables
print "variables keys: \t\n", f.variables.keys()

dimension keys: 	
['lon_0', 'lat_0']
dimensions: 	
{'lon_0': 7000, 'lat_0': 3500}
variables: 	
{'lon_0': <Nio.NioVariable object at 0x10b2b6090>, 'PrecipRate_P0_L102_GLL0': <Nio.NioVariable object at 0x10b2b6b50>, 'lat_0': <Nio.NioVariable object at 0x10b2b6550>}
variables keys: 	
['lon_0', 'PrecipRate_P0_L102_GLL0', 'lat_0']


## Read in variables as numpy arrays

In [10]:
precip  = f.variables["PrecipRate_P0_L102_GLL0"]                          
lat = f.variables["lat_0"][:]       # [:] reads the values                            
lon = f.variables["lon_0"][:]

## Print some info about the vars

In [11]:
print "precip type is",type(precip)
print "lat type is %s" % type(lat)
print "lon type is %s" % type(lon)
print "lat dtype is %s" % lat.dtype

precip type is <class 'Nio.NioVariable'>
lat type is <type 'numpy.ndarray'>
lon type is <type 'numpy.ndarray'>
lat dtype is float32


## Accessing individual attributes and dimension names

In [12]:
print "Variable is %s (%s)" % (precip.long_name,precip.units)
print "Name of dimensions is", precip.dimensions

Variable is Radar Precipitation Rate (mm/hr)
Name of dimensions is ('lat_0', 'lon_0')


## Use [:] to access values

In [13]:
precipvals = precip[:]             # Reads all dimensions, all values
print "precipvals type",type(precipvals)
print "precipvals shape",precipvals.shape
print "precipvals dtype = %s" % precipvals.dtype

precipvals type <class 'numpy.ma.core.MaskedArray'>
precipvals shape (3500, 7000)
precipvals dtype = float32


In [14]:
print precipvals

[[-3. -3. -3. ..., -3. -3. -3.]
 [-3. -3. -3. ..., -3. -3. -3.]
 [-3. -3. -3. ..., -3. -3. -3.]
 ..., 
 [-3. -3. -3. ..., -3. -3. -3.]
 [-3. -3. -3. ..., -3. -3. -3.]
 [-3. -3. -3. ..., -3. -3. -3.]]


In [15]:
# Two ways to get the min/max
# numpy doesn't handle missing values well, so generally use the second one
print "precipvals min / max = %g / %g" % (np.min(precipvals),np.max(precipvals))
print "precipvals min / max = %g / %g" % (precipvals.min(),precipvals.max())

precipvals min / max = -3 / 103.8
precipvals min / max = -3 / 103.8


In [16]:
# Two ways to calculate simple average
# these should be the same, unless you have a masked array
print "precipvals average = ",np.mean(precipvals)   # Be careful with this!
print "precipvals average = ",precipvals.mean()

precipvals average =  -0.999979913235
precipvals average =  -0.999979913235


In [17]:
## It looks like -3. is empty? Restrict to greater than or equal to zero
print "precipvals min / max = %g / %g" % (precipvals[precipvals>=0.].min(),precipvals[precipvals>=0.].max())

precipvals min / max = 0 / 103.8


In [18]:
print "precipvals average = ",precipvals[precipvals>=0.].mean()

precipvals average =  0.0815964341164


In [20]:
print precip._FillValue

[  1.00000002e+20]
